# Transformer

## 필요한 library import

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

## SelfAttention 클래스

`SelfAttention` 클래스는 Transformer 아키텍처의 핵심 구성 요소입니다. 주어진 시퀀스에 대한 attention 점수를 계산하고 attention 메커니즘의 주 출력인 값들의 가중치 합을 반환합니다.

### 초기화

```python
def __init__(self, embed_size, heads):
```

- `embed_size`: 시퀀스의 각 토큰에 대한 임베딩 크기.
- `heads`: attention 헤드의 수. 여러 헤드를 사용하면 모델이 입력의 다른 부분에 동시에 집중할 수 있습니다.

초기화 내부에서:

1. 임베딩 크기는 헤드 수로 나누어 각 헤드의 크기를 결정합니다.
2. 값, 키, 쿼리를 위한 선형 계층이 정의됩니다. 이들은 입력 시퀀스를 각각의 표현으로 변환합니다.
3. 모든 헤드의 출력을 결합하기 위한 최종 완전 연결(fc) 계층이 정의됩니다.

### 순방향 전달

```python
def forward(self, values, keys, query, mask):
```

- `values`, `keys`, `query`: 이들은 입력 시퀀스가 각각의 표현으로 변환된 것입니다.
- `mask`: 일반적으로 패딩 토큰을 위해 사용되는 값들을 마스킹하는 데 사용됩니다.

단계:

1. 입력 시퀀스는 헤드를 분리하기 위해 재구성됩니다.
2. 시퀀스는 그들의 각각의 선형 계층을 통과합니다.
3. 스케일링 된 내적 attention은 다음과 같이 계산됩니다: $\text{attention} = \frac{\text{queries} \times \text{keys}^T}{\sqrt{\text{head\_dim}}}$
4. 마스크가 제공되면, attention 점수는 마스크에 따라 조정됩니다.
5. attention 점수는 softmax 함수를 통해 정규화됩니다.
6. 출력은 attention 가중치와 값의 내적으로 계산되며, 모든 헤드의 출력을 결합하기 위해 재구성됩니다.

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(SelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads

        assert (
            self.head_dim * heads == embed_size
        ), "Embedding size needs to be divisible by heads"

        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        # Split the embedding into self.heads different pieces
        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        values = self.values(values)
        keys = self.keys(keys)
        queries = self.queries(queries)

        # Scaled dot-product attention
        attention = torch.einsum("nqhd,nkhd->nhqk", [queries, keys]) / math.sqrt(self.head_dim)
        if mask is not None:
            attention = attention.masked_fill(mask == 0, float("-1e20"))

        attention = F.softmax(attention, dim=3)

        out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
            N, query_len, self.heads * self.head_dim
        )

        out = self.fc_out(out)
        return out

## TransformerBlock 클래스

`TransformerBlock` 클래스는 Transformer 아키텍처의 기본 블록을 나타냅니다. 이 클래스는 Self-Attention 메커니즘과 Feed Forward 신경망을 결합하여 구성됩니다.

### 초기화

```python
def __init__(self, embed_size, heads, dropout, forward_expansion):
```

- `embed_size`: 시퀀스의 각 토큰에 대한 임베딩 크기.
- `heads`: attention 헤드의 수.
- `dropout`: 드롭아웃 비율.
- `forward_expansion`: Feed Forward 신경망의 확장 계수.

초기화 내부에서:

1. `SelfAttention` 메커니즘을 초기화합니다.
2. 입력 데이터의 정규화를 위한 두 개의 Layer Normalization 계층을 정의합니다.
3. Feed Forward 신경망을 정의합니다. 이 신경망은 두 개의 선형 계층으로 구성되며, 활성화 함수로 ReLU를 사용합니다.
4. 드롭아웃 계층을 정의합니다.

### 순방향 전달

```python
def forward(self, value, key, query, mask):
```

- `value`, `key`, `query`: Self-Attention 메커니즘에 필요한 입력 값입니다.
- `mask`: 패딩 토큰을 위한 마스크.

단계:

1. 입력 값은 `SelfAttention` 메커니즘을 통과하여 attention 값을 얻습니다.
2. 얻은 attention 값과 원래의 query 값을 더하여 skip connection을 구현합니다.
3. 결과는 첫 번째 Layer Normalization과 드롭아웃을 통과합니다.
4. 그 다음, Feed Forward 신경망을 통과합니다.
5. Feed Forward 신경망의 출력과 이전의 출력을 더하여 또 다른 skip connection을 구현합니다.
6. 마지막으로, 두 번째 Layer Normalization과 드롭아웃을 통과하여 최종 출력을 얻습니다.

In [ ]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(TransformerBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)

        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, value, key, query, mask):
        attention = self.attention(value, key, query, mask)

        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + query))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(EncoderBlock, self).__init__()
        self.attention = SelfAttention(embed_size, heads)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attention = self.attention(x, x, x, mask)
        # Add skip connection, run through normalization and finally dropout
        x = self.dropout(self.norm1(attention + x))
        forward = self.feed_forward(x)
        out = self.dropout(self.norm2(forward + x))
        return out

## Encoder 클래스

`Encoder` 클래스는 Transformer 아키텍처의 인코더 부분을 나타냅니다. 이 클래스는 주어진 입력 시퀀스를 처리하여 연속적인 표현을 생성합니다.

### 초기화

```python
def __init__(
    self,
    src_vocab_size,
    embed_size,
    num_layers,
    heads,
    device,
    forward_expansion,
    dropout,
    max_length,
):
```

- `src_vocab_size`: 소스 어휘의 크기.
- `embed_size`: 시퀀스의 각 토큰에 대한 임베딩 크기.
- `num_layers`: Transformer 블록의 수.
- `heads`: attention 헤드의 수.
- `device`: 연산을 수행할 장치 (예: "cuda" 또는 "cpu").
- `forward_expansion`: Feed Forward 신경망의 확장 계수.
- `dropout`: 드롭아웃 비율.
- `max_length`: 입력 시퀀스의 최대 길이.

초기화 내부에서:

1. 단어 임베딩과 위치 임베딩을 정의합니다.
2. 지정된 수의 `TransformerBlock` 레이어를 초기화하여 모듈 리스트에 추가합니다.
3. 드롭아웃 계층을 정의합니다.

### 순방향 전달

```python
def forward(self, x, mask):
```

- `x`: 입력 시퀀스.
- `mask`: 패딩 토큰을 위한 마스크.

단계:

1. 입력 시퀀스의 길이와 배치 크기를 가져옵니다.
2. 입력 시퀀스의 각 토큰에 대한 위치를 계산합니다.
3. 단어 임베딩과 위치 임베딩을 합하여 최종 임베딩을 얻습니다. 이후 드롭아웃을 적용합니다.
4. 최종 임베딩은 모든 `TransformerBlock` 레이어를 순차적으로 통과합니다.
5. 최종 출력을 반환합니다.

In [ ]:
class Encoder(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        embed_size,
        num_layers,
        heads,
        device,
        forward_expansion,
        dropout,
        max_length,
    ):

        super(Encoder, self).__init__()
        self.embed_size = embed_size
        self.device = device
        self.word_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                TransformerBlock(
                    embed_size,
                    heads,
                    dropout=dropout,
                    forward_expansion=forward_expansion,
                )
                for _ in range(num_layers)
            ]
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        out = self.dropout(
            (self.word_embedding(x) + self.position_embedding(positions))
        )

        for layer in self.layers:
            out = layer(out, out, out, mask)

        return out


## DecoderBlock 클래스

`DecoderBlock` 클래스는 Transformer 아키텍처의 디코더 부분의 핵심 구성 요소입니다. 이 클래스는 인코더의 출력과 함께 타겟 시퀀스를 처리하여 최종 출력을 생성합니다.

### 초기화

```python
def __init__(self, embed_size, heads, forward_expansion, dropout, device):
```

- `embed_size`: 시퀀스의 각 토큰에 대한 임베딩 크기.
- `heads`: attention 헤드의 수.
- `forward_expansion`: Feed Forward 신경망의 확장 계수.
- `dropout`: 드롭아웃 비율.
- `device`: 연산을 수행할 장치 (예: "cuda" 또는 "cpu").

초기화 내부에서:

1. 입력 데이터의 정규화를 위한 Layer Normalization 계층을 정의합니다.
2. Self-Attention 메커니즘을 초기화합니다.
3. `TransformerBlock` 레이어를 초기화합니다.
4. 드롭아웃 계층을 정의합니다.

### 순방향 전달

```python
def forward(self, x, value, key, src_mask, trg_mask):
```

- `x`: 디코더의 입력 시퀀스.
- `value`, `key`: 인코더의 출력 값.
- `src_mask`: 소스 패딩 토큰을 위한 마스크.
- `trg_mask`: 타겟 패딩 토큰을 위한 마스크.

단계:

1. 입력 `x`는 Self-Attention 메커니즘을 통과하여 attention 값을 얻습니다.
2. 얻은 attention 값과 원래의 `x` 값을 더하여 skip connection을 구현합니다. 결과는 Layer Normalization과 드롭아웃을 통과합니다.
3. 이후, `TransformerBlock` 레이어를 통과하여 최종 출력을 얻습니다.

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_size, heads, forward_expansion, dropout, device):
        super(DecoderBlock, self).__init__()
        self.norm = nn.LayerNorm(embed_size)
        self.attention = SelfAttention(embed_size, heads)
        self.transformer_block = TransformerBlock(
            embed_size, heads, dropout, forward_expansion
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, value, key, src_mask, trg_mask):
        attention = self.attention(x, x, x, trg_mask)
        query = self.dropout(self.norm(attention + x))
        out = self.transformer_block(value, key, query, src_mask)
        return out

In [ ]:
class Decoder(nn.Module):
    def __init__(
        self,
        trg_vocab_size,
        embed_size,
        num_layers,
        heads,
        forward_expansion,
        dropout,
        device,
        max_length,
    ):
        super(Decoder, self).__init__()
        self.device = device
        self.word_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)

        self.layers = nn.ModuleList(
            [
                DecoderBlock(embed_size, heads, forward_expansion, dropout, device)
                for _ in range(num_layers)
            ]
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_out, src_mask, trg_mask):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length).expand(N, seq_length).to(self.device)
        x = self.dropout((self.word_embedding(x) + self.position_embedding(positions)))

        for layer in self.layers:
            x = layer(x, enc_out, enc_out, src_mask, trg_mask)

        out = self.fc_out(x)
        return out

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        trg_pad_idx,
        embed_size=512,
        num_layers=6,
        forward_expansion=4,
        heads=8,
        dropout=0,
        device="cuda",
        max_length=100,
    ):

        super(Transformer, self).__init__()

        self.encoder = Encoder(
            src_vocab_size,
            embed_size,
            num_layers,
            heads,
            device,
            forward_expansion,
            dropout,
            max_length,
        )

        self.decoder = Decoder(
            trg_vocab_size,
            embed_size,
            num_layers,
            heads,
            forward_expansion,
            dropout,
            device,
            max_length,
        )

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device

    def make_src_mask(self, src):
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return src_mask.to(self.device)

    def make_trg_mask(self, trg):
        N, trg_len = trg.shape
        trg_mask = torch.tril(torch.ones((trg_len, trg_len))).expand(
            N, 1, trg_len, trg_len
        )
        return trg_mask.to(self.device)

    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        enc_src = self.encoder(src, src_mask)
        out = self.decoder(trg, enc_src, src_mask, trg_mask)
        return out

In [ ]:
import torch.optim as optim

# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 0.0003
SRC_VOCAB_SIZE = 5000  # Example value, adjust as needed
TRG_VOCAB_SIZE = 5000  # Example value, adjust as needed
SRC_PAD_IDX = 0
TRG_PAD_IDX = 0

# Create a random dataset for demonstration purposes
# Normally, you'd load your actual dataset here
def create_data(n_samples=1000, max_length=50, src_vocab_size=5000, trg_vocab_size=5000):
    src_data = torch.randint(0, src_vocab_size - 1, (n_samples, max_length))
    trg_data = torch.randint(0, trg_vocab_size - 1, (n_samples, max_length))
    return src_data, trg_data

src_data, trg_data = create_data()

# Initialize the Transformer model
model = Transformer(
    SRC_VOCAB_SIZE, TRG_VOCAB_SIZE, SRC_PAD_IDX, TRG_PAD_IDX
).to("cuda")

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

# Training loop
for epoch in range(EPOCHS):
    for batch_idx in range(0, len(src_data), BATCH_SIZE):
        src = src_data[batch_idx: batch_idx + BATCH_SIZE].to("cuda")
        trg = trg_data[batch_idx: batch_idx + BATCH_SIZE].to("cuda")

        optimizer.zero_grad()
        output = model(src, trg[:, :-1])
        output = output.reshape(-1, output.shape[2])
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item()}")

print("Training finished!")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, random_split

# Dataset and DataLoader
dataset = TensorDataset(src_data, trg_data)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Training and validation loop
for epoch in range(EPOCHS):
    # Training
    model.train()
    total_loss = 0
    for src, trg in train_loader:
        src, trg = src.to("cuda"), trg.to("cuda")

        optimizer.zero_grad()
        output = model(src, trg[:, :-1])
        output = output.reshape(-1, output.shape[2])
        trg = trg[:, 1:].reshape(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch: {epoch}, Training Loss: {total_loss / len(train_loader)}")

    # Validation
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for src, trg in val_loader:
            src, trg = src.to("cuda"), trg.to("cuda")
            output = model(src, trg[:, :-1])
            output = output.reshape(-1, output.shape[2])
            trg = trg[:, 1:].reshape(-1)
            loss = criterion(output, trg)
            total_loss += loss.item()

    print(f"Epoch: {epoch}, Validation Loss: {total_loss / len(val_loader)}")

# Save the model
torch.save(model.state_dict(), "transformer_model.pth")
print("Model saved!")

# Load the model (if needed later)
model.load_state_dict(torch.load("transformer_model.pth"))
print("Model loaded!")